# Qrack Quantum Fourier Transform Validation

We'd like a little more assurance that Qrack's performance reflects _accurate_ behavior on the QFT. If we can trust another simulator as a "gold standard," _regardless of convention_, this should be as simple as running identical circuits on Qrack and the "gold standard" and calculating the _inner product between output state vectors_, which should be equal to 1.

Mind you, even "Big Tech" software has bugs _all the time_, including quantum computer simulator software! (Dan has _found_ them in "Big Tech" and other quantum open source software, in the past, but, once identified, they tend to get fixed.) It's not a given, that _any_ simulator should be trusted as a perfect "gold standard" simply for the size of the organization that develops it! However, if we go ahead and run this notebook, it's reassuring to see that, up to floating-point error, we _do_ see an inner product of 1 between Qrack's default optimal stack and Qiskit Aer's state vector simulator.

In [1]:
low = 1
high = 12
samples = 10

In [2]:
import random
import math
import numpy as np

In [3]:
%env QRACK_QUNITMULTI_DEVICES 1
from pyqrack import QrackSimulator, Pauli


env: QRACK_QUNITMULTI_DEVICES=1


### Random circuit initialization

In [4]:
#RCS gates

def rand_u3(sim, q):
    th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    lm = random.uniform(0, 4 * math.pi)
    sim.u(th, ph, lm, q)

def cx(sim, q1, q2):
    sim.cx(q1, q2)

def cy(sim, q1, q2):
    sim.cy(q1, q2)

def cz(sim, q1, q2):
    sim.cz(q1, q2)

def acx(sim, q1, q2):
    sim.x(q1)
    sim.cx(q1, q2)
    sim.x(q1)

def acy(sim, q1, q2):
    sim.x(q1)
    sim.cy(q1, q2)
    sim.x(q1)

def acz(sim, q1, q2):
    sim.x(q1)
    sim.cz(q1, q2)
    sim.x(q1)

def swap(circ, q1, q2):
    circ.swap(q1, q2)

two_bit_gates = cx, cz, cy, acx, acz, acy

In [5]:
from qiskit import QuantumCircuit
from qiskit import execute, Aer
from qiskit.providers.aer import QasmSimulator

def reverse(num_qubits, sim):
    start = 0
    end = num_qubits - 1
    while (start < end):
        sim.swap(start, end)
        start += 1
        end -= 1

# Implementation of the Quantum Fourier Transform
# (See https://qiskit.org/textbook/ch-algorithms/quantum-fourier-transform.html)
def aer_qft(n, circuit):
    if n == 0:
        return circuit
    n -= 1

    circuit.h(n)
    for qubit in range(n):
        circuit.cp(math.pi/2**(n-qubit), qubit, n)

    # Recursive QFT is very similiar to a ("classical") FFT
    aer_qft(n, circuit)

sim_backend = QasmSimulator(shots=1, method='statevector_gpu')

inner_product_results = {}
for n in range(low, high + 1):
    sim = QrackSimulator(n)
    sim.set_reactive_separate(False)
    
    width_results = []
        
    # Run the benchmarks
    for i in range(samples):
        sim.reset_all()
        circ = QuantumCircuit(n, n)
        
        for i in range(n):
            # Single bit gates
            for j in range(n):
                rand_u3(circ, j)

            # Multi bit gates
            bit_set = [i for i in range(n)]
            while len(bit_set) > 1:
                b1 = random.choice(bit_set)
                bit_set.remove(b1)
                b2 = random.choice(bit_set)
                bit_set.remove(b2)
                gate = random.choice(two_bit_gates)
                gate(circ, b1, b2)

        # Qrack can directly parse the Qiskit circuit,
        # and there's basically no better guarantee we've implemented this correctly,
        # than running exactly the same gates.
        sim.run_qiskit_circuit(circ)
        
        # Qubit order and QFT-vs.-inverse convention are just convention,
        # but we also know that sim.qft() does its job correctly in Shor's, (elsewhere).
        sim.iqft([i for i in reversed(range(n))])
        reverse(n, sim)

        qrack_sv = sim.out_ket()

        aer_qft(n, circ)
        reverse(n, circ)

        circ.save_statevector()
        job = execute([circ], sim_backend)
        aer_sv = np.asarray(job.result().get_statevector())
        
        width_results.append(np.abs(sum([np.conj(x)*y for x,y in zip(qrack_sv,aer_sv)])))

    inner_product_results[n] = sum(width_results) / samples

print(inner_product_results)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
{1: 0.9999999990781255, 2: 0.9999999630890555, 3: 0.999999958544136, 4: 0.9999998972522585, 5: 0.9999999278268061, 6: 0.9999998620306906, 7: 0.9999997709445652, 8: 0.9999998206455174, 9: 0.999999798359138, 10: 0.9999996560480673, 11: 0.9999997303562494, 12: 0.9999996416416984}
